In [5]:
import os

import numpy as np
import pandas as pd
from pandas.testing import assert_frame_equal

from hbmep.config import Config
from hbmep.model import BaseModel

from paper.utils import load_rat_data, setup_logging

DIR = "/mount/hdd1/acute_mapping/proc/physio"
SUBJECTS = range(1, 9)

BUILD_DIR = "/home/vishu/data/hbmep-processed/rat"
os.makedirs(BUILD_DIR, exist_ok=True)


In [6]:
unique_subdirs = []

for subject in SUBJECTS:
    curr_dir = os.path.join(DIR, f"amap{subject:002}")
    unique_subdirs += os.listdir(curr_dir)

unique_subdirs = list(set(unique_subdirs))
len(unique_subdirs)


FileNotFoundError: [Errno 2] No such file or directory: '/mount/hdd1/acute_mapping/proc/physio/amap01'

## J device

    1. J_RCML_000   -- J_RCML

    2. J_SHAP_000   -- J_SHAP
    3. J_SHAP_001   -- J_SHAP



In [ ]:
j_dirs = [d for d in unique_subdirs if "J" in d]
len(j_dirs)


32

In [ ]:
j_rcml_dirs = [d for d in j_dirs if "RCML" in d]
print(len(j_rcml_dirs))

j_rcml_dirs


16


['2023-03-13_J_RCML_000',
 '2023-04-03_J_RCML_002_mid',
 '2023-04-04_J_RCML_003_J7_reduced',
 '2023-04-04_J_RCML_002_J1_lateral',
 '2023-04-03_J_RCML_003_norm',
 '2023-03-27_J_RCML_000',
 '2023-04-03_J_RCML_001_lat',
 '2023-03-17_J_RCML_000',
 '2023-04-04_J_RCML_004_J7_midline',
 '2023-03-29_J_RCML_000',
 '2023-04-03_J_RCML_000',
 '2023-03-24_J_RCML_000',
 '2023-04-04_J_RCML_001_J1_midline',
 '2023-03-21_J_RCML_000',
 '2023-04-04_J_RCML_005_J7_lateral',
 '2023-04-04_J_RCML_000']

In [ ]:
j_shap_dirs = [d for d in j_dirs if "SHAP" in d]
print(len(j_shap_dirs))

j_shap_dirs


16


['2023-03-29_J_SHAP_001',
 '2023-03-24_J_SHAP_001',
 '2023-03-27_J_SHAP_001',
 '2023-03-21_J_SHAP_000',
 '2023-03-27_J_SHAP_000',
 '2023-03-17_J_SHAP_000',
 '2023-03-13_J_SHAP_000',
 '2023-04-03_J_SHAP_000',
 '2023-03-21_J_SHAP_001',
 '2023-04-03_J_SHAP_001',
 '2023-03-29_J_SHAP_000',
 '2023-04-04_J_SHAP_000',
 '2023-03-24_J_SHAP_000',
 '2023-04-04_J_SHAP_001',
 '2023-03-13_J_SHAP_001',
 '2023-03-17_J_SHAP_001']

### J_RCML data

In [ ]:
subdir_pattern = ["*J_RCML_000*"]

df, mat, time, auc_window, muscles_map = load_rat_data(
    dir=DIR, subdir_pattern=subdir_pattern, subjects=SUBJECTS
)


100%|█████████████████████████████████████████████████████████| 8/8 [00:02<00:00,  3.02it/s]


In [ ]:
src = "/home/vishu/data/hbmep-processed/rat/J_RCML/data.csv"
comp_df = pd.read_csv(src)
comp_df = comp_df.rename(columns=muscles_map).copy()

src = "/home/vishu/data/hbmep-processed/rat/J_RCML/mat.npy"
comp_mat = np.load(src)

assert_frame_equal(df, comp_df)
assert (mat == comp_mat).all()


In [ ]:
def plot_dataset(df, toml_path, build_dir, mat=None):
    config = Config(toml_path=toml_path)
    config.BUILD_DIR = build_dir
    model = BaseModel(config)
    df = df.copy()
    df, encoder_dict = model.preprocess(df)
    model.plot(df=df, encoder_dict=encoder_dict)
    return


In [ ]:
# toml_path = "/home/vishu/repos/rat-mapping-paper/configs/J_RCML.toml"
# build_dir = "/home/vishu/data/hbmep-processed/rat/J_RCML"
# plot_dataset(df, toml_path, build_dir)


2024-07-09 10:03:39.140011: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.0 which is older than the ptxas CUDA version (12.5.40). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


KeyboardInterrupt: 

### J_SHAP data

In [ ]:
subdir_pattern = ["*J_SHAP*"]

df, mat, time, auc_window, muscles_map = load_rat_data(
    dir=DIR, subdir_pattern=subdir_pattern, subjects=SUBJECTS
)

df.pulse_amplitude = (
    df
    .apply(
        lambda x: (x.pulse_amplitude * 4) if x.compound_charge_params == "80-0-20-400" else x.pulse_amplitude,
        axis=1
    )
)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:07<00:00,  1.12it/s]


In [ ]:
src = "/home/vishu/data/hbmep-processed/rat/J_SHAP/data.csv"
comp_df = pd.read_csv(src)

src = "/home/vishu/data/hbmep-processed/rat/J_SHAP/mat.npy"
comp_mat = np.load(src)

assert_frame_equal(df, comp_df)
assert (mat == comp_mat).all()


In [ ]:
# toml_path = "/home/vishu/repos/rat-mapping-paper/configs/J_SHAP.toml"
# build_dir = "/home/vishu/data/hbmep-processed/rat/J_SHAP"
# plot_dataset(df, toml_path, build_dir)


### J_SHAP_000 data

In [10]:
subdir_pattern = ["*J_SHAP_000*"]

df, mat, time, auc_window, muscles_map = load_rat_data(
    dir=DIR, subdir_pattern=subdir_pattern, subjects=SUBJECTS
)

df.pulse_amplitude = (
    df
    .apply(
        lambda x: (x.pulse_amplitude * 4) if x.compound_charge_params == "80-0-20-400" else x.pulse_amplitude,
        axis=1
    )
)


100%|█████████████████████████████████████████████████████████| 8/8 [00:02<00:00,  2.81it/s]


In [11]:
dest = "/home/vishu/data/hbmep-processed/rat//data.csv"
df.to_csv(dest, index=False)

dest = "/home/vishu/data/hbmep-processed/rat/J_SHAP_000/mat.npy"
np.save(dest, mat)


### L_CIRC data

In [ ]:
[d for d in unique_subdirs if "L_CIRC" in d]


['2023-03-29_L_SHIE_000',
 '2023-04-04_L_SHIE_000',
 '2023-03-17_L_SHIE_000',
 '2023-03-24_L_SHIE_000',
 '2023-03-13_L_SHIE_000',
 '2023-03-21_L_SHIE_000',
 '2023-03-27_L_SHIE_000',
 '2023-04-03_L_SHIE_000',
 '2023-03-17_L_SHIE_001']

In [ ]:
subdir_pattern = ["*L_CIRC*"]

df, mat, time, auc_window, muscles_map = load_rat_data(
    dir=DIR, subdir_pattern=subdir_pattern, subjects=SUBJECTS
)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  4.02it/s]


In [ ]:
src = "/home/vishu/data/hbmep-processed/rat/L_CIRC/data.csv"
comp_df = pd.read_csv(src)

src = "/home/vishu/data/hbmep-processed/rat/L_CIRC/mat.npy"
comp_mat = np.load(src)

assert_frame_equal(df, comp_df)
assert (mat == comp_mat).all()


### L_SHIE data

In [7]:
[d for d in unique_subdirs if "L_SHIE" in d]


['2023-03-29_L_SHIE_000',
 '2023-04-04_L_SHIE_000',
 '2023-03-17_L_SHIE_000',
 '2023-03-24_L_SHIE_000',
 '2023-03-13_L_SHIE_000',
 '2023-03-21_L_SHIE_000',
 '2023-03-27_L_SHIE_000',
 '2023-04-03_L_SHIE_000',
 '2023-03-17_L_SHIE_001']

In [8]:
subdir_pattern = ["*L_SHIE*"]

df, mat, time, auc_window, muscles_map = load_rat_data(
    dir=DIR, subdir_pattern=subdir_pattern, subjects=SUBJECTS
)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  4.02it/s]


In [9]:
src = "/home/vishu/data/hbmep-processed/rat/L_SHIE/data.csv"
comp_df = pd.read_csv(src)

src = "/home/vishu/data/hbmep-processed/rat/L_SHIE/mat.npy"
comp_mat = np.load(src)

assert_frame_equal(df, comp_df)
assert (mat == comp_mat).all()
